In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import linregress
from sklearn.cluster import KMeans
from PIL import Image, ImageDraw
from skimage.color import rgb2gray
from skimage.draw import disk
from matplotlib.patches import Circle
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from pathlib import Path
import re

# Functions

In [3]:
def key_from_csv(p: Path) -> str:
    """
    C2...nd2_(series_01)_0233-0247.csv  ->  C2...nd2_(series_01)
    """
    name = p.stem  # no .csv
    # remove trailing _####-#### (or similar) if present
    name = re.sub(r"_\d+-\d+$", "", name)
    return name

def key_from_img(p: Path) -> str:
    """
    C2...nd2_(series_01).jpg -> C2...nd2_(series_01)
    """
    return p.stem  # no .jpg


In [ ]:
def scatter_plot(df, x_col, y_col, 
                 x_lim=None, 
                 y_lim=None):

    plt.figure(figsize=(4, 4), dpi=150)
    plt.scatter(df[x_col], df[y_col], alpha=0.6)

    plt.xlabel(x_col.replace("_", " "))
    plt.ylabel(y_col.replace("_", " "))

    # Fixed scale
    if x_lim is not None:
        plt.xlim(x_lim)

    if y_lim is not None:
        plt.ylim(y_lim)

    plt.tight_layout()
    plt.show()



def plot_histogram(df, column, bins=50, xlabel=None, title=None,
                   figsize=(4, 3), dpi=200):
    """
    Plot histogram for a dataframe column.

    Parameters:
    df : pandas.DataFrame
    column : str
        Column name to plot
    bins : int
        Number of bins
    xlabel : str (optional)
    title : str (optional)
    """

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)

    ax.hist(
        df[column].dropna(),
        bins=bins,
        edgecolor="black",
        linewidth=0.5,
        alpha=0.8
    )

    ax.set_xlabel(xlabel if xlabel else column, fontsize=11)
    ax.set_ylabel("Count", fontsize=11)
    ax.set_title(title if title else f"Distribution of {column}", fontsize=12)

    # Clean style
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.tight_layout()
    plt.show()


def plot_foci_on_image(
    gray_image,
    df,
    x_col="x_px",
    y_col="y_px",
    r_col="sigma_px",
    circle_color="red",
    center_size=6,
    linewidth=1,
    figsize=(7, 7),
    show=True
):
    """
    Plot detected foci as circles on a grayscale image.

    Parameters
    ----------
    gray_image : 2D numpy array
        Grayscale image.
    df : pandas DataFrame
        DataFrame containing coordinates and radii.
    x_col, y_col : str
        Column names for x and y coordinates.
    r_col : str
        Column name for radius (in pixels).
    circle_color : str
        Color of circle and center dot.
    center_size : int
        Size of center dot.
    linewidth : int
        Circle outline thickness.
    figsize : tuple
        Figure size.
    show : bool
        Whether to call plt.show().
    """

    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(gray_image, cmap="gray")

    for x_px, y_px, r_px in zip(df[x_col], df[y_col], df[r_col]):
        # circle outline
        ax.add_patch(
            Circle(
                (x_px, y_px),
                r_px,
                fill=False,
                edgecolor=circle_color,
                linewidth=linewidth
            )
        )
        # center dot
        ax.scatter(x_px, y_px, c=circle_color, s=center_size)

    ax.axis("off")

    if show:
        plt.show()

    return fig, ax



In [32]:
dir_nuclei_stat = "./examples"
dir_images = "./examples"
dir_foci = "./examples/run"

In [33]:
nuclei_path = Path(str(dir_nuclei_stat).strip()) # path to data about nucleus in total
images_path = Path(str(dir_images).strip())
foci_path = Path(str(dir_foci).strip()) # path to data about foci in the particular nucleus


In [ ]:
nuclei_files = sorted(nuclei_path.glob("*.csv"))
dfs1 = [] 
images_paths = []

for f in nuclei_files:
        # find the corresponding image
        image_name = f.stem.replace("_roi", "") + ".tif"
        image_path = images_path.with_name(image_name)
        images.append(image_path)

        key = key_from_csv(f)
        key = key[:-4]
        df = pd.read_csv(f)
        df.columns = df.columns.str.strip()  # remove hidden spaces in headers

        # ensure expected columns exist
        if "Area" not in df.columns or "Mean" not in df.columns:
            raise KeyError(
                f"In nuclei file {f.name} expected columns 'Area' and 'Mean'. "
                f"Found: {list(df.columns)}"
            )
        
        df["File_name"] = key
        df = df.rename(columns={"Area": "Nucleus_area", "Mean": "Nucleus_MFI"})
        df = df[["File_name", "Nucleus_area", "Nucleus_MFI"]]
        dfs1.append(df)

    final = pd.concat(dfs1, ignore_index=True)

In [41]:
for name in final["File_name"]:
    print(name)

C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(series_01)
C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(series_02)
C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(series_03)


In [42]:
final

,File_name,Nucleus_area,Nucleus_MFI
0,C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(seri...,249.198,18.703
1,C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(seri...,249.445,13.178
2,C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(seri...,280.680,9.722


# Data import

## Filtration based on sigma value

In [ ]:
df = df[(df["sigma [nm]"] > 75)]
print("Number of foci after filtration: ", df.shape[0])


## Calculate number of outliers based on MFI of foci

In [ ]:
data = df["mean_intensity"]
mean = data.mean()
std = data.std()
Q1 = np.percentile(data, 25)
Q3 = np.percentile(data, 75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

outliers = df[df["mean_intensity"] > upper_bound]
print("Number of outliers: ", outliers.shape[0])

In [ ]:
# Plot
plt.figure(figsize=(6,4))
plt.hist(data, bins=50, alpha=0.6)
plt.axvline(upper_bound, linestyle="--", linewidth=2)

plt.xlabel("Mean intensity")
plt.ylabel("Frequency")
plt.title("")

plt.show()

## Draw circle area

In [ ]:
df = pd.read_csv("./examples/run/C2_MP_U2OS_fixed_20nMJF549_ORC1_MGS1.nd2_(series_01)_0199-0257.csv")

In [ ]:
px_size_ts_x = 11.6
px_size_ts_y = 11.6

px_size_x = 57.5
px_size_y = 58.7

sx = px_size_ts_x/px_size_x
sy = px_size_ts_y/px_size_y
ssigma = np.mean([px_size_ts_x, px_size_ts_y]) / np.mean([px_size_x, px_size_y])


df["x_px"] = sx*df["x_nm"] / px_size_ts_x
df["y_px"] = sy*df["y_nm"] / px_size_ts_y

# For sigma (use average pixel size)
df["sigma_px"] = ssigma*df["sigma_nm"] / np.mean([px_size_ts_x, px_size_ts_y])

In [ ]:
def compute_mean_intensity_from_localizations(
        image_path,
        df,
        px_size_ts_x = 11.6,
        px_size_ts_y = 11.6,
        px_size_x = 57.5,
        px_size_y = 58.7,
        x_col="x [nm]",
        y_col="y [nm]",
        sigma_col="sigma [nm]"
    ):

        # Open image
        image = Image.open(image_path).convert("RGB")

        # Convert image to grayscale
        gray = rgb2gray(image)
        H, W = gray.shape # number of pixels

        # Scaling factors
        sx = px_size_ts_x/px_size_x
        sy = px_size_ts_y/px_size_y
        ssigma = np.mean([px_size_ts_x, px_size_ts_y]) / np.mean([px_size_x, px_size_y])

        # Storage lists
        x_list = []
        y_list = []
        sigma_list = []
        mean_list = []

        for _, row in df.iterrows():
            x_nm = row[x_col]
            y_nm = row[y_col]
            sigma_nm = row[sigma_col]

            # original pixels → current image pixels
            x_px = int(round(sx * x_nm / px_size_ts_x))
            y_px = int(round(sy * y_nm / px_size_ts_y))
            sigma_px = max(1, int(round(ssigma * sigma_nm / np.mean([px_size_ts_x, px_size_ts_y])))) # minimal possible value is 1 pixel!

            # Build circular mask (clipped automatically)
            rr, cc = disk((y_px, x_px), sigma_px, shape=(H, W))
            mask = np.zeros((H, W), dtype=bool)
            mask[rr, cc] = True
            disk((y_px, x_px), sigma_px, shape=(H, W))

            # Compute mean intensity
            if mask.sum() > 0:
                mean_intensity = gray[mask].mean()
            else:
                mean_intensity = np.nan

            x_list.append(x_px)
            y_list.append(y_px)
            sigma_list.append(sigma_px)
            mean_list.append(mean_intensity)

        # Return modified copy
        df_out = df.copy()
        df_out["x_px"] = x_list
        df_out["y_px"] = y_list
        df_out["sigma_px"] = sigma_list
        df_out["mean_intensity"] = mean_list

        return df_out

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(gray, cmap="gray")

for x_px, y_px, r_px in zip(res["x_px"], res["y_px"], res["sigma_px"]):
    # circle outline
    ax.add_patch(Circle((x_px, y_px), r_px, fill=False, edgecolor="red", linewidth=1))
    # center dot
    ax.scatter(x_px, y_px, c="red", s=6)

ax.axis("off")
plt.show()

# Kmean

In [ ]:
n_clusters = 3

# Select TWO columns
X = df[["mean_intensity", "sigma [nm]"]].dropna()

# Run KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X)

# Add cluster labels back to dataframe
df.loc[X.index, "cluster"] = labels

centers = kmeans.cluster_centers_

print("Cluster centers:")
for i, c in enumerate(centers):
    print(f"Cluster {i}: mean_intensity={c[0]:.2f}, sigma={c[1]:.2f}")

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(5, 4), dpi=200)

# Scatter plot of clusters
for cluster_id in range(n_clusters):
    cluster_data = df[df["cluster"] == cluster_id]
    plt.scatter(cluster_data["sigma [nm]"],
                cluster_data["mean_intensity"],
                alpha=0.6,
                label=f"Cluster {cluster_id}")

# Plot cluster centers
plt.scatter(centers[:, 1],   # sigma
            centers[:, 0],   # mean_intensity
            marker="X",
            s=200,
            linewidths=2,
            edgecolors="black",
            label="Centers")

plt.xlabel("Sigma")
plt.ylabel("Mean intensity")
plt.title("KMeans Clustering (Sigma vs Intensity)")
plt.legend()

# Clean look
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
plot_histogram(df, column="sigma [nm]", xlabel="Sigma (nm)", title="Distribution of Sigma")

Task:
1. sigma - 2 populations (KMeans)
2. Outliers of sigma - check them:
 - big bright dots: analyze as a third population
 - aggregates: analyze separatly
 - MFI of nucleus - big dots/aggregates
3. How the bg susbstruction affects the sigma, MFI of foci?

In [ ]:
means = df.groupby("sigma [nm]")["mean_intensity"].mean()
plt.bar(means.index, means.values)
plt.xlabel("sigma [nm]")
plt.ylabel("Mean intensity")
plt.xticks(rotation=45)
plt.show()

# Filtration

In [ ]:
df_bright = df[df["sigma [nm]"] > 400]
df_medium = df[(df["sigma [nm]"] > 150) & (df["sigma [nm]"] <= 400)]
df_dark = df[df["sigma [nm]"] <= 150]

print("Bright foci count:", df_bright.shape[0])
print("Medium foci count:", df_medium.shape[0])
print("Dark foci count:", df_dark.shape[0])

In [ ]:
df1 = df[df["sigma [nm]"] > 500]
print("Bright foci count:", df1.shape[0])